In [397]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.formula.api as smf
import statsmodels.api as sm
from lib import utils
from lib import firth

In [462]:
# Load data
df_choice = pd.read_csv('choice_data.csv',sep=',',index_col=0)

df_choice['part_control'] = (df_choice['group_value'] == False)*df_choice['part_value']
df_choice['part_treatment'] = (df_choice['group_value'] == True)*df_choice['part_value']

# Identify the obs with no change in choice
sum_choice_seq = df_choice.groupby('worker_id')['choice_value'].sum().to_frame()
analysis_workers = sum_choice_seq[(sum_choice_seq['choice_value'] != 24) & (sum_choice_seq['choice_value'] != 0)].index
all_seq_workers = sum_choice_seq[(sum_choice_seq['choice_value'] == 24)].index
all_single_workers = sum_choice_seq[(sum_choice_seq['choice_value'] == 0)].index

# Adjust worker_id
# Always choosing sequence: -88
# ALways choosing single: -99
df_choice['all_single'] = df_choice['worker_id'].isin(all_single_workers)
df_choice['all_seq'] = df_choice['worker_id'].isin(all_seq_workers)
df_choice['adj_worker_id'] = df_choice['worker_id'].isin(analysis_workers) * df_choice['worker_id'] + \
                                df_choice['worker_id'].isin(all_single_workers) * (-99) + \
                                df_choice['worker_id'].isin(all_seq_workers) * (-88)

# Create dummies
worker_dummies = pd.get_dummies(df_choice['worker_id'], prefix='worker')
front_ratio_dummies = pd.get_dummies(df_choice['front_ratio'], prefix='front_ratio')
single_amount_dummies = pd.get_dummies(df_choice['single_amount'], prefix='single_amount')
df_choice = pd.concat([df_choice,worker_dummies,front_ratio_dummies,single_amount_dummies],axis=1)

cols_worker = [i for i in worker_dummies if i!= worker_dummies.columns[0]]
cols_front_ratio = [i for i in front_ratio_dummies if i!= front_ratio_dummies.columns[0]]
cols_single_amount = [i for i in single_amount_dummies if i!= single_amount_dummies.columns[0]]

# Covert boolean vairables to numerical variables
bool_cols = df_choice.select_dtypes(include=['bool']).columns
df_choice[bool_cols] = df_choice[bool_cols].astype(int)

df_analysis = df_choice[df_choice['worker_id'].isin(analysis_workers)]

x_cols = ['group_value','part_control','part_treatment']
cols_worker_analysis = [c for c in cols_worker if int(c.split('_')[1]) in analysis_workers]

wrong_workers = df_rabbit['worker_id'][df_rabbit['choice_correct'] == 0].unique()
cols_worker_wrong = [c for c in cols_worker if int(c.split('_')[1]) in wrong_workers]



In [460]:
# Pooled regression
y = df_choice['choice_value']
X = sm.add_constant(df_choice[x_cols + cols_front_ratio + cols_single_amount])
mod = sm.Logit(y,X, data=df_choice)
result_1 = mod.fit(cov_type='cluster',cov_kwds={'groups':df_choice['worker_id']})
result_1.summary()

Optimization terminated successfully.
         Current function value: 0.680128
         Iterations 4


d:\Python\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['data']
  warnings.warn(msg, ValueWarning)
d:\Python\lib\site-packages\statsmodels\base\model.py:127: ValueWarning: unknown kwargs ['data']
  warnings.warn(msg, ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           choice_value   No. Observations:                 7056
Model:                          Logit   Df Residuals:                     7044
Method:                           MLE   Df Model:                           11
Date:                Wed, 17 Apr 2024   Pseudo R-squ.:                 0.01727
Time:                        03:12:52   Log-Likelihood:                -4799.0
converged:                       True   LL-Null:                       -4883.3
Covariance Type:              cluster   LLR p-value:                 2.230e-30
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.2907      0.149     -1.947      0.051      -0.583       0.002
group_value           0.0817      0.189      0.432      0.666      -0.289       0.453
part_control          0.0845      0.059      1.434      0.152      -0.031       0.200
part_treatment        0.2226      0.067      3.305      0.001       0.091       0.355
front_ratio_0.2      -0.2758      0.051     -5.383      0.000      -0.376      -0.175
front_ratio_0.3      -0.3331      0.059     -5.623      0.000      -0.449      -0.217
front_ratio_0.4      -0.2536      0.061     -4.158      0.000      -0.373      -0.134
front_ratio_0.5      -0.1929      0.071     -2.708      0.007      -0.332      -0.053
front_ratio_0.6      -0.4677      0.081     -5.763      0.000      -0.627      -0.309
single_amount_240     0.1906      0.044      4.373      0.000       0.105       0.276
single_amount_280     0.6654      0.058     11.474      0.000       0.552       0.779
single_amount_320     0.4753      0.053      8.954      0.000       0.371       0.579
=====================================================================================
"""

In [461]:
# Fixed-effect regression
y = df_analysis['choice_value']
X = sm.add_constant(df_analysis[x_cols + cols_front_ratio + cols_single_amount + cols_worker_analysis])
mod = sm.Logit(y,X)
result_2 = mod.fit(cov_type='cluster',cov_kwds={'groups':df_analysis['worker_id']})
# result_2.summary()

Optimization terminated successfully.
         Current function value: 0.442363
         Iterations 7


In [463]:
# Fixed-effect regression
y = df_choice['choice_value']
X = sm.add_constant(df_choice[x_cols + cols_front_ratio + cols_single_amount + cols_worker_analysis + ['all_single','all_seq']])
mod = sm.Logit(y,X)
result_3 = mod.fit(cov_type='cluster',cov_kwds={'groups':df_choice['adj_worker_id']})
# result_3.summary()

         Current function value: 0.277042
         Iterations: 35


d:\Python\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [323]:
# Firth regression
y = df_choice['choice_value']
X = sm.add_constant(df_choice[x_cols + cols_front_ratio + cols_single_amount + cols_worker])
firth_reg_1 = firth.firthLogit(y,X)
firth_reg_1.fit()

iteration: 0 , LL= 4690.526436155515


iteration: 1 , LL= 2268.6921749153566
iteration: 2 , LL= 1955.691791822022
iteration: 3 , LL= 1866.1166776184036
iteration: 4 , LL= 1839.2422350079216
iteration: 5 , LL= 1830.6266410158762
iteration: 6 , LL= 1828.1744393287115
iteration: 7 , LL= 1827.7026538469495
iteration: 8 , LL= 1827.666835551051
iteration: 9 , LL= 1827.6664093264396
iteration: 10 , LL= 1827.6663999132204
iteration: 11 , LL= 1827.666398128969
iteration: 12 , LL= 1827.6663977293206
iteration: 13 , LL= 1827.6663976359328
iteration: 14 , LL= 1827.6663976138893
iteration: 15 , LL= 1827.6663976086738
iteration: 16 , LL= 1827.6663976074392
iteration: 17 , LL= 1827.6663976071472
iteration: 18 , LL= 1827.6663976070777
iteration: 19 , LL= 1827.6663976070613
iteration: 20 , LL= 1827.6663976070574
iteration: 21 , LL= 1827.6663976070565
iteration: 22 , LL= 1827.6663976070565


In [464]:
firth_reg_1.clusterSE(cluster_var=df_choice['adj_worker_id'])
wald_result_1 = firth_reg_1.wald(use_cluster=True)
wald_coef_result_1 = wald_result_1[wald_result_1['var_name'].isin(['const'] + x_cols + cols_front_ratio + cols_single_amount)]
wald_coef_result_1.to_csv('firth_reg_result.csv')

Confidence level:  0.95


In [524]:
def draw_reg_col(result,col_name,var_names=None,digit=3):
    
    if var_names is None:
        var_names = result.params.index
    
    col_result = pd.DataFrame(columns=[col_name])

    for r in range(len(var_names)):
        _var = var_names[r]
        _param = str(round(result.params.loc[_var],digit)) + utils.get_star(result.pvalues.loc[_var])
        _se = '(' + str(round(result.bse.loc[_var],digit)) +')'
        col_result.loc['b_'+_var] = _param
        col_result.loc['se_'+_var] = _se

    col_result.loc['nobs'] = int(result.nobs)
    col_result.loc['aic'] = str(round(result.aic,digit))

    return col_result

def draw_reg_firth(result,col_name,var_names=None,digit=3):
    
    if var_names is None:
        var_names = result.index
    
    col_result = pd.DataFrame(columns=[col_name])

    for r in range(len(var_names)):
        _var = var_names[r]
        _param = str(round(result.coef.loc[_var],digit)) + utils.get_star(result.p_value.loc[_var])
        _se = '(' + str(round(result.bse.loc[_var],digit)) + ')'
        col_result.loc['b_'+_var] = _param
        col_result.loc['se_'+_var] = _se

    return col_result

In [528]:
reg_col_1 = draw_reg_col(result_1,col_name='(1) Pooled',var_names=result_1.params.index[1:])
reg_col_2 = draw_reg_col(result_2,col_name='(2) FE',var_names=result_1.params.index[1:])
reg_col_3 = draw_reg_col(result_3,col_name='(3) FE',var_names=result_1.params.index[1:])

firth_result = wald_result_1.set_index('var_name')
reg_col_firth = draw_reg_firth(firth_result,col_name='(3) Firth',var_names=result_1.params.index)

# reg_cols = reg_col_1.join([reg_col_2, reg_col_3])
# reg_cols.at['nobs','(3) Firth'] = len(y)
reg_cols = reg_col_1.join([reg_col_2,reg_col_3], how='outer')
reg_cols = reg_cols.reindex(reg_col_3.index).fillna('')
reg_cols

,(1) Pooled,(2) FE,(3) FE
b_group_value,0.082,0.034,-0.1
se_group_value,(0.189),(0.102),(0.096)
b_part_control,0.084,0.289,0.289
se_part_control,(0.059),(0.185),(0.183)
b_part_treatment,0.223$^{***}$,0.535$^{***}$,0.535$^{***}$
se_part_treatment,(0.067),(0.162),(0.161)
b_front_ratio_0.2,-0.276$^{***}$,-0.756$^{***}$,-0.756$^{***}$
se_front_ratio_0.2,(0.051),(0.135),(0.133)
b_front_ratio_0.3,-0.333$^{***}$,-0.915$^{***}$,-0.915$^{***}$
se_front_ratio_0.3,(0.059),(0.156),(0.155)


In [529]:
param_list = ['Group',
            '',
            r'Question$\cdot1\{\text{Group}=0\}$',
            '',
            r'Question$\cdot1\{\text{Group}=1\}$',
            '',
            r'$1\{\rho=0.2\}$',
            '',
            r'$1\{\rho=0.3\}$',
            '',
            r'$1\{\rho=0.4\}$',
            '',
            r'$1\{\rho=0.5\}$',
            '',
            r'$1\{\rho=0.6\}$',
            '',
            r'$1\{s=240\}$',
            '',
            r'$1\{s=280\}$',
            '',
            r'$1\{s=320\}$',
            '',
            'observations',
            'aic']


reg_cols.index = param_list
utils.make_table(reg_cols,'tables/reg_choice.tex')

In [468]:
# Load data
df_rabbit = pd.read_csv('rabbit_data.csv',sep=',',index_col=0)

df_rabbit['part_control'] = (df_rabbit['group_value'] == False)*df_rabbit['part_value']
df_rabbit['part_treatment'] = (df_rabbit['group_value'] == True)*df_rabbit['part_value']
 
# df_analysis = df_choice[df_choice['worker_id'].isin(analysis_workers)]

# Create dummy variables
worker_dummies = pd.get_dummies(df_rabbit['worker_id'], prefix='worker')
front_amount_dummies = pd.get_dummies(df_rabbit['front_amount'], prefix='front_amount')
single_amount_dummies = pd.get_dummies(df_rabbit['single_amount'], prefix='single_amount')
diff_amount_dummies = pd.get_dummies(df_rabbit['diff_amount'], prefix='diff_amount')
df_rabbit = pd.concat([df_rabbit,worker_dummies,front_amount_dummies,single_amount_dummies,diff_amount_dummies],axis=1)

cols_worker = [i for i in worker_dummies if i!= worker_dummies.columns[0]]
cols_front_rabbit = [i for i in front_amount_dummies if i!= front_amount_dummies.columns[0]]
cols_single_rabbit = [i for i in single_amount_dummies if i!= single_amount_dummies.columns[0]]
cols_diff_rabbit = [i for i in diff_amount_dummies if i!= diff_amount_dummies.columns[0]]

# Covert boolean vairables to numerical variables
bool_cols = df_rabbit.select_dtypes(include=['bool']).columns
df_rabbit[bool_cols] = df_rabbit[bool_cols].astype(int)

# Create regression sample
df_rabbit_analysis = df_rabbit[df_rabbit['worker_id'].isin(analysis_workers)]
df_rabbit_wrong = df_rabbit[df_rabbit['worker_id'].isin(wrong_workers)]

In [469]:
y = df_rabbit['choice_value']
X = sm.add_constant(df_rabbit[x_cols + cols_diff_rabbit + cols_front_rabbit + cols_single_rabbit])
reg_rabbit_1 = sm.Logit(y,X).fit(cov_type='cluster',cov_kwds={'groups':df_rabbit['worker_id']})
reg_rabbit_1.summary()

Optimization terminated successfully.
         Current function value: 0.123110
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           choice_value   No. Observations:                 3504
Model:                          Logit   Df Residuals:                     3496
Method:                           MLE   Df Model:                            7
Date:                Wed, 17 Apr 2024   Pseudo R-squ.:                  0.8222
Time:                        03:14:42   Log-Likelihood:                -431.38
converged:                       True   LL-Null:                       -2426.5
Covariance Type:              cluster   LLR p-value:                     0.000
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -4.3312      0.309    -14.026      0.000      -4.936      -3.726
group_value        -0.7296      0.330     -2.208      0.027      -1.377      -0.082
part_control        0.5738      0.226      2.539      0.011       0.131       1.017
part_treatment      0.0624      0.318      0.196      0.844      -0.561       0.686
diff_amount_1       7.7970      0.377     20.681      0.000       7.058       8.536
front_amount_2      0.1060      0.274      0.387      0.699      -0.431       0.643
front_amount_3     -0.6546      0.228     -2.876      0.004      -1.101      -0.209
single_amount_8     0.1451      0.166      0.876      0.381      -0.180       0.470
===================================================================================
"""

In [470]:
y = df_rabbit_analysis['choice_value']
X = sm.add_constant(df_rabbit_analysis[x_cols + cols_diff_rabbit + cols_front_rabbit + cols_single_rabbit + cols_worker_analysis])
reg_rabbit_2 = sm.Logit(y,X).fit(cov_type='cluster',cov_kwds={'groups':df_rabbit_analysis['worker_id']})
# reg_rabbit_2.summary()

Optimization terminated successfully.
         Current function value: 0.083734
         Iterations 10


In [480]:
y = df_rabbit['choice_value']
X = sm.add_constant(df_rabbit[x_cols + cols_diff_rabbit + cols_front_rabbit + cols_single_rabbit + cols_worker])
reg_rabbit_3 = sm.Logit(y,X).fit(cov_type='cluster',cov_kwds={'groups':df_rabbit['worker_id']})
# reg_rabbit_2.summary()

         Current function value: 0.071438
         Iterations: 35


d:\Python\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [481]:
y = df_rabbit_wrong['choice_value']
X = sm.add_constant(df_rabbit_wrong[x_cols + cols_diff_rabbit + cols_front_rabbit + cols_single_rabbit + cols_worker_wrong])
reg_rabbit_4 = sm.Logit(y,X).fit(cov_type='cluster',cov_kwds={'groups':df_rabbit_wrong['worker_id']})
# reg_rabbit_3.summary()


         Current function value: 0.269047
         Iterations: 35


d:\Python\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [530]:
rabbit_col_1 = draw_reg_col(reg_rabbit_1,col_name='(1) Pooled',var_names=reg_rabbit_1.params.index[1:])
rabbit_col_2 = draw_reg_col(reg_rabbit_2,col_name='(2) FE',var_names=reg_rabbit_1.params.index[1:])
rabbit_col_3 = draw_reg_col(reg_rabbit_3,col_name='(3) FE',var_names=reg_rabbit_1.params.index[1:])
rabbit_col_4 = draw_reg_col(reg_rabbit_4,col_name='(4) FE',var_names=reg_rabbit_1.params.index[1:])

rabbit_cols = rabbit_col_1.join([rabbit_col_2,rabbit_col_3,rabbit_col_4], how='outer')
rabbit_cols

,(1) Pooled,(2) FE,(3) FE,(4) FE
b_group_value,-0.73$^{*}$,-4.379$^{***}$,-4.379$^{***}$,-0.757
se_group_value,(0.33),(1.114),(1.114),(0.392)
b_part_control,0.574$^{*}$,1.378$^{*}$,1.378$^{*}$,2.005$^{**}$
se_part_control,(0.226),(0.6),(0.6),(0.744)
b_part_treatment,0.062,0.152,0.152,0.145
se_part_treatment,(0.318),(0.368),(0.368),(0.344)
b_diff_amount_1,7.797$^{***}$,11.71$^{***}$,11.71$^{***}$,6.181$^{***}$
se_diff_amount_1,(0.377),(1.091),(1.091),(0.552)
b_front_amount_2,0.106,0.183,0.183,0.217
se_front_amount_2,(0.274),(0.378),(0.378),(0.393)


In [531]:
param_list = ['Group',
            '',
            r'Question$\cdot1\{\text{Group}=0\}$',
            '',
            r'Questsion$\cdot1\{\text{Group}=1\}$',
            '',
            r'$1\{r_2 + r_3 > r_1\}$',
            '',
            r'$1\{r_2 =2\}$',
            '',
            r'$1\{r_2=3\}$',
            '',
            r'$1\{r_1=8\}$',
            '',
            'observations',
            'aic']


rabbit_cols.index = param_list
utils.make_table(rabbit_cols,'tables/reg_rabbit.tex')

In [492]:
df_analysis['predict_choice'] = (result_2.predict() > 0.5).astype(int)
# df_analysis['predict_choice'] = firth_reg_1.predict()
outlier_threshold = df_analysis['response_time'].quantile(0.995)
df_response = df_analysis[df_analysis['response_time'] < outlier_threshold]

# create interactions
df_response['i_choice_group'] = df_response['predict_choice']*df_response['group_value']
df_response['i_choice_control'] = df_response['predict_choice']*df_response['part_control']
df_response['i_choice_treat'] = df_response['predict_choice']*df_response['part_treatment']


cols_i_front_ratio = []
cols_i_single_amount = []

for r in range(len(cols_front_ratio)):
    new_col = 'i_choice_'+ cols_front_ratio[r]
    df_response[new_col] = df_response['predict_choice'] * df_response[cols_front_ratio[r]]
    cols_i_front_ratio += [new_col]

for r in range(len(cols_single_amount)):
    new_col = 'i_choice_'+ cols_single_amount[r]
    df_response[new_col] = df_response['predict_choice'] * df_response[cols_single_amount[r]]
    cols_i_single_amount += [new_col]

C:\Users\zarkwang\AppData\Local\Temp\ipykernel_8432\4274444507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis['predict_choice'] = (result_2.predict() > 0.5).astype(int)
C:\Users\zarkwang\AppData\Local\Temp\ipykernel_8432\4274444507.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_response['i_choice_group'] = df_response['predict_choice']*df_response['group_value']
C:\Users\zarkwang\AppData\Local\Temp\ipykernel_8432\4274444507.py:8: SettingWithCopyWarning: 
A value is trying to be set 

In [493]:
df_rabbit['predict_choice'] = (reg_rabbit_1.predict() > 0.5).astype(int)
# df_analysis['predict_choice'] = firth_reg_1.predict()
outlier_threshold = df_rabbit['response_time'].quantile(0.995)
df_response_rabbit = df_rabbit[df_rabbit['response_time'] < outlier_threshold]

# create interactions
df_response_rabbit['i_choice_group'] = df_response_rabbit['predict_choice']*df_response_rabbit['group_value']
df_response_rabbit['i_choice_control'] = df_response_rabbit['predict_choice']*df_response_rabbit['part_control']
df_response_rabbit['i_choice_treat'] = df_response_rabbit['predict_choice']*df_response_rabbit['part_treatment']


cols_i_front_rabbit = []
cols_i_single_rabbit = []
cols_i_diff_rabbit = []

for r in range(len(cols_front_rabbit)):
    new_col = 'i_choice_'+ cols_front_rabbit[r]
    df_response_rabbit[new_col] = df_response_rabbit['predict_choice'] * df_response_rabbit[cols_front_rabbit[r]]
    cols_i_front_rabbit += [new_col]

for r in range(len(cols_single_rabbit)):
    new_col = 'i_choice_'+ cols_front_rabbit[r]
    df_response_rabbit[new_col] = df_response_rabbit['predict_choice'] * df_response_rabbit[cols_single_rabbit[r]]
    cols_i_single_rabbit += [new_col]

for r in range(len(cols_diff_rabbit)):
    new_col = 'i_choice_'+ cols_diff_rabbit[r]
    df_response_rabbit[new_col] = df_response_rabbit['predict_choice'] * df_response_rabbit[cols_diff_rabbit[r]]
    cols_i_diff_rabbit += [new_col]

C:\Users\zarkwang\AppData\Local\Temp\ipykernel_8432\2906315383.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_response_rabbit['i_choice_group'] = df_response_rabbit['predict_choice']*df_response_rabbit['group_value']
C:\Users\zarkwang\AppData\Local\Temp\ipykernel_8432\2906315383.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_response_rabbit['i_choice_group'] = df_response_rabbit['predict_choice']*df_response_rabbit['group_value']
C:\Users\zarkwang\AppData\Local\Temp\ipykernel_8432\2906315383.py:8: Perf

In [510]:
y = df_response_rabbit['response_time']/1000
x_cols_new = x_cols + ['predict_choice','i_choice_group','i_choice_control','i_choice_treat']
X = sm.add_constant(df_response_rabbit[x_cols_new + cols_diff_rabbit + cols_i_diff_rabbit + \
                                cols_front_rabbit + cols_i_front_rabbit + \
                                cols_single_rabbit + cols_i_single_rabbit + cols_worker])

mod_2nd_rabbit = sm.OLS(y,X).fit(cov_type='cluster',cov_kwds={'groups':df_response_rabbit['worker_id']})

In [515]:
def draw_reg_col(result,col_name,var_names=None,digit=3):
    
    if var_names is None:
        var_names = result.params.index
    
    col_result = pd.DataFrame(columns=[col_name])

    for r in range(len(var_names)):
        _var = var_names[r]
        _param = str(round(result.params.loc[_var],digit)) + utils.get_star(result.pvalues.loc[_var])
        _se = '(' + str(round(result.bse.loc[_var],digit)) +')'
        col_result.loc['b_'+_var] = _param
        col_result.loc['se_'+_var] = _se

    col_result.loc['nobs'] = int(result.nobs)
    col_result.loc['aic'] = str(round(result.aic,digit))
    col_result.loc['adj_r'] = str(round(result.rsquared_adj,digit))

    return col_result

response_col_choice = draw_reg_col(mod_2nd,col_name='Intertemporal Choice',var_names=x_cols_new)
response_col_rabbit = draw_reg_col(mod_2nd_rabbit,col_name='Rabbit',var_names=x_cols_new)
response_cols = response_col_choice.join([response_col_rabbit], how='outer')
response_cols

,Intertemporal Choice,Rabbit
b_group_value,-0.706$^{***}$,-2.438$^{***}$
se_group_value,(0.144),(0.118)
b_part_control,-0.102,0.787$^{***}$
se_part_control,(0.166),(0.15)
b_part_treatment,0.414$^{***}$,0.784$^{***}$
se_part_treatment,(0.108),(0.109)
b_predict_choice,0.434,0.499$^{***}$
se_predict_choice,(0.336),(0.063)
b_i_choice_group,-0.69$^{*}$,-1.17$^{***}$
se_i_choice_group,(0.3),(0.199)


In [518]:
param_list = ['Group',
            '',
            r'Question$\cdot1\{\text{Group}=0\}$',
            '',
            r'Questsion$\cdot1\{\text{Group}=1\}$',
            '',
            'Choice',
            '',
            r'Choice$\times$Group',
            '',
            r'Choice$\times$Question$\cdot1\{\text{Group}=0\}$',
            '',
            r'Choice$\times$Question$\cdot1\{\text{Group}=1\}$',
            '',
            'observations',
            'aic',
            r'adj-$R^2$']


response_cols.index = param_list
utils.make_table(response_cols,'tables/reg_response_time.tex')